# explaining the functions used to run the model

### load_mat
Purpose: Load data from a .mat file.
Inputs:
name: The name of the .mat file.
split_real_imaginary: If True, splits the real and imaginary parts of the data.
include_metadata: List of additional metadata to include.
Outputs: A tuple of tensors containing the loaded data.

In [ ]:
def load_mat(
    name: str,
    split_real_imaginary: bool = True,
    include_metadata: List[Literal["fiber_fractions"]] = []
    ) -> Tuple[Tensor, Tensor]:
    # Function to load .mat file and return tensor data
    pass


### load_point_clouds
Purpose: Load point cloud data from a file.
Inputs:
name: The name of the file containing point cloud data.
Outputs: A tuple of dictionaries containing the point clouds.

In [ ]:
def load_point_clouds(name: str) -> Tuple[Dict[int, Tensor], Dict[int, Tensor]]:
    # Function to load point cloud data from a file
    pass


### get_mags
Purpose: Convert a complex tensor to a magnitude tensor.
Inputs:
voxel: A tensor containing complex values.
Outputs: A tensor containing the magnitudes of the complex values

### visualize
Purpose: Visualize voxel data.
Inputs:
voxel: A tensor containing voxel data.
int_range: Intensity range for visualization.
title: Title of the plot.
Outputs: None. This function visualizes the data.

In [ ]:
def visualize(voxel, int_range=[0.6,1], title='Plot'):
    # Function to visualize voxel data
    pass


### VoxelDataset(Dataset)
Purpose: Custom dataset class for handling voxel data and corresponding lines.
Inputs:
voxel_data: Tensor containing voxel data.
lines: Tensor containing line data (ground truth from Hough Transform).
Outputs: Provides methods to get the length of the dataset and to get individual data samples.

In [ ]:
class VoxelDataset(Dataset):
    def __init__(self, voxel_data, lines):
        self.voxel_data = voxel_data
        self.lines = lines

    def __len__(self):
        return len(self.voxel_data)

    def __getitem__(self, idx):
        return self.voxel_data[idx], self.lines[idx]


### HoughNN
Purpose: Define a 3D CNN model for voxel data.
Inputs:
x: Input tensor of shape (batch_size, 1, 33, 33, 33).
Outputs:
Output tensor of shape (batch_size, 6) representing the line parameters [npoints, a1, a2, a3, b1, b2, b3]

In [ ]:
class HoughNN(nn.Module):
    def __init__(self):
        super(HoughNN, self).__init__()
        self.conv1 = nn.Conv3d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv3d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool3d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 6)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



### train_model
Purpose: Train the neural network model.
Inputs:
model: The neural network model to train.
dataloader: DataLoader providing the training data.
criterion: Loss function.
optimizer: Optimizer for training.
num_epochs: Number of epochs to train.
Outputs: Trained model.

In [ ]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in dataloader:
            inputs = inputs.unsqueeze(1).float()  # Add channel dimension
            targets = targets.float()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Epoch {epoch}/{num_epochs}, Loss: {epoch_loss:.4f}')

    return model


### evaluate_model
Purpose: Evaluate the neural network model.
Inputs:
model: The neural network model to evaluate.
dataloader: DataLoader providing the evaluation data.
criterion: Loss function.
Outputs: Mean Squared Error (MSE) of the model on the evaluation data

In [ ]:
def evaluate_model(model, dataloader, criterion):
    model.eval()
    mse = 0.0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.unsqueeze(1).float()
            targets = targets.float()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            mse += loss.item() * inputs.size(0)

    mse /= len(dataloader.dataset)
    return mse
